In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 71.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [2]:
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from transformers import Trainer, TrainingArguments
from transformers import VisionEncoderDecoderModel, AutoProcessor
from datasets import load_dataset
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm_notebook as tqdm
from transformers import AdamW
import torch.nn.functional as F
device='cuda' if torch.cuda.is_available() else 'cpu'

In [146]:
# Load the model and processor
model = VisionEncoderDecoderModel.from_pretrained("qantev/trocr-small-spanish")
processor = AutoProcessor.from_pretrained("qantev/trocr-small-spanish")
tokenizer = TrOCRProcessor.from_pretrained('qantev/trocr-small-spanish')


In [147]:
model.load_state_dict(torch.load("/kaggle/input/trocr-small/pytorch/default/1/model_weights_grayscale_contrast.pth"))

/tmp/ipykernel_30/446732016.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/trocr-small/pytorch/default/1/model_weights_g

<All keys matched successfully>

In [81]:
path = "/kaggle/input/extra-spanish-text-1k/train_texts.csv"
df1 = pd.read_csv(path)

In [82]:
df1.columns.values[0] = 'unique Id'  # Change the first column heading to "unique Id"
df1 = df1.rename(columns={'transcription': 'nans'})
df1 = df1.rename(columns={'unique Id': 'transcription'})

In [83]:
df1["unique Id"] = range(len(df1))

In [84]:
df1

,transcription,nans,unique Id
transcription_0,"to, sin saber tres virtud tiene. Y",NaN,0
transcription_1,"tos juicios de darme tiempo, para",NaN,1
transcription_2,que en vuestra educación lo mani,NaN,2
transcription_3,"sestalle, pues me hallo tantos meses",NaN,3
transcription_4,"ha rendida en esta cama, a una en",NaN,4
...,...,...,...
transcription_996,"en sus tierras, en sus naciones.",NaN,996
transcription_997,"También le nacieron hijos á Sem,",NaN,997
transcription_998,de todos los hijos de,NaN,998
transcription_999,estos fueron hijos de Joctán.,NaN,999


In [53]:
path1 = "/kaggle/input/ai-of-god-3/Public_data/train.csv"
ai_train = pd.read_csv(path1)

In [54]:
ai_train

,unique Id,transcription
0,Rodrigo_00006_00,Historia De España_Del
1,Rodrigo_00006_01,Arçobispo. Do Rodri_
2,Rodrigo_00006_02,go. Traducida En Ro_
3,Rodrigo_00006_03,mançe
4,Rodrigo_00008_00,E ste es el libro de la Cronica de es
...,...,...
15005,Rodrigo_00620_19,mes En zelo de la fe & partian lo que Auian a ...
15006,Rodrigo_00620_20,an menester. & se parauan muy rezios En los pe...
15007,Rodrigo_00620_21,suffrian mucho trabajo por amor de dios / del ...
15008,Rodrigo_00620_22,"tilla estos prelados, don Rodrigo Arçobispo de..."


In [55]:
train_df = df1

In [56]:
import re
from collections import Counter

# Function to clean and tokenize transcriptions
def tokenize_transcription(transcription):
    # Lowercase the text, remove punctuation, and split by whitespace
    transcription = transcription.lower()
    words = re.findall(r'\b\w+\b', transcription)  # Extract words
    return words

# Create a list to store all words
all_words = []
transcription = train_df["transcription"]
transcription1 = ai_train["transcription"]
# Iterate through your dataset and tokenize each transcription

for line in transcription:
    words = tokenize_transcription(line)
    all_words.extend(words)

for line in transcription1:
    words = tokenize_transcription(line)
    all_words.extend(words)
# Count the frequency of each word
word_counts = Counter(all_words)

In [57]:
load_old_spanish_dictionary = list(word_counts.keys())

# Print some information
print(f"Total unique words: {len(load_old_spanish_dictionary)}")

Total unique words: 13336


In [58]:
from transformers import BertTokenizer
from collections import defaultdict

# Load BERT Spanish tokenizer
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

# Extract the BERT vocabulary (token IDs mapped to words)
bert_vocab = set(tokenizer.get_vocab().keys())

# Use the Old Spanish dictionary as a set (assuming load_old_spanish_dictionary is a list)
old_spanish_dictionary = set(load_old_spanish_dictionary)  # Use the list directly

# Union of BERT and Old Spanish dictionaries
combined_vocabulary = bert_vocab.union(old_spanish_dictionary)

# Check some words from combined vocabulary
print(f"Size of Combined Vocabulary: {len(combined_vocabulary)}")


Size of Combined Vocabulary: 41476


In [59]:
test_df = pd.read_csv('/kaggle/input/ai-of-god-3/Public_data/test.csv')

In [60]:
test_path = '/kaggle/input/ai-of-god-3/Public_data/test_images'

# Function to convert unique ID into correct image path
def construct_image_path(unique_id):
    parts = unique_id.split('_')
    page_number = parts[1]
    l_number = parts[3] 
    
    image_path = os.path.join(test_path, f'Page_{page_number}', f'L_{l_number}.png')
    return image_path

test_df['image_path'] = test_df['unique Id'].apply(construct_image_path)

# Check the resulting dataframe
print(test_df[[ 'image_path']].head())

                                          image_path
0  /kaggle/input/ai-of-god-3/Public_data/test_ima...
1  /kaggle/input/ai-of-god-3/Public_data/test_ima...
2  /kaggle/input/ai-of-god-3/Public_data/test_ima...
3  /kaggle/input/ai-of-god-3/Public_data/test_ima...
4  /kaggle/input/ai-of-god-3/Public_data/test_ima...


In [64]:
extra_df = pd.read_csv("/kaggle/input/extra-spanish-text-1k/train_texts.csv")

In [69]:
extra_df

,unique Id,transcription
transcription_0,"to, sin saber tres virtud tiene. Y",NaN
transcription_1,"tos juicios de darme tiempo, para",NaN
transcription_2,que en vuestra educación lo mani,NaN
transcription_3,"sestalle, pues me hallo tantos meses",NaN
transcription_4,"ha rendida en esta cama, a una en",NaN
...,...,...
transcription_996,"en sus tierras, en sus naciones.",NaN
transcription_997,"También le nacieron hijos á Sem,",NaN
transcription_998,de todos los hijos de,NaN
transcription_999,estos fueron hijos de Joctán.,NaN


In [80]:
df1

,transcription,nans,unique Id,image_path
transcription_0,"to, sin saber tres virtud tiene. Y",NaN,0,/kaggle/input/extra-spanish-text-1k/train_imag...
transcription_1,"tos juicios de darme tiempo, para",NaN,1,/kaggle/input/extra-spanish-text-1k/train_imag...
transcription_2,que en vuestra educación lo mani,NaN,2,/kaggle/input/extra-spanish-text-1k/train_imag...
transcription_3,"sestalle, pues me hallo tantos meses",NaN,3,/kaggle/input/extra-spanish-text-1k/train_imag...
transcription_4,"ha rendida en esta cama, a una en",NaN,4,/kaggle/input/extra-spanish-text-1k/train_imag...
...,...,...,...,...
transcription_996,"en sus tierras, en sus naciones.",NaN,996,/kaggle/input/extra-spanish-text-1k/train_imag...
transcription_997,"También le nacieron hijos á Sem,",NaN,997,/kaggle/input/extra-spanish-text-1k/train_imag...
transcription_998,de todos los hijos de,NaN,998,/kaggle/input/extra-spanish-text-1k/train_imag...
transcription_999,estos fueron hijos de Joctán.,NaN,999,/kaggle/input/extra-spanish-text-1k/train_imag...


In [86]:
extra_path = '/kaggle/input/extra-spanish-text-1k/train_images/train_images'
# Base directory for the images
base_image_path = '/kaggle/input/extra-spanish-text-1k/train_images/train_images/'

# Loop over the dataset to create image paths
image_paths = []

for i, row in df1.iterrows():
    # Construct the image path based on the unique ID
    image_path = f"{base_image_path}transcription_{row['unique Id']}.png"
    image_paths.append(image_path)

# Add the image paths to your dataframe
df1['image_path'] = image_paths

# Check the first few entries to confirm the paths
print(df1[['unique Id', 'image_path']].head())


                 unique Id                                         image_path
transcription_0          0  /kaggle/input/extra-spanish-text-1k/train_imag...
transcription_1          1  /kaggle/input/extra-spanish-text-1k/train_imag...
transcription_2          2  /kaggle/input/extra-spanish-text-1k/train_imag...
transcription_3          3  /kaggle/input/extra-spanish-text-1k/train_imag...
transcription_4          4  /kaggle/input/extra-spanish-text-1k/train_imag...


In [61]:
from torchvision import transforms

In [62]:
model.to(device)

VisionEncoderDecoderModel(
  (encoder): DeiTModel(
    (embeddings): DeiTEmbeddings(
      (patch_embeddings): DeiTPatchEmbeddings(
        (projection): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DeiTEncoder(
      (layer): ModuleList(
        (0-11): 12 x DeiTLayer(
          (attention): DeiTAttention(
            (attention): DeiTSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): DeiTSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): DeiTIntermediate(
            (dense): Linear(

In [ ]:
# predictions = []

# model.eval()

# with torch.no_grad():
#     for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Inference"):
#         image_path = row['image_path']
        
#         # Open and preprocess the image (convert to grayscale, normalize, etc.)
#         image = Image.open(image_path).convert("RGB")
        
#         # Apply the transformations (grayscale, normalize, and ToTensor)
#         transform = transforms.Compose([
#             transforms.Grayscale(num_output_channels=3),
#             transforms.ToTensor()
#         ])
        
#         image = transform(image)

#         image_g = image.to(device)

#         mask = image_g < 0.6875
#         image_g[mask] = 0.0
#         image_g[~mask] = 1.0
        
#         # Process the image using the processor
#         pixel_values = processor(image_g, return_tensors="pt", do_rescale=False).pixel_values.to(device)

#         # Generate output
#         outputs = model.generate(pixel_values)

#         # Decode the predicted string
#         predicted_string = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
#         # Append the results
#         predictions.append({
#             'unique Id': row['unique Id'],  # Ensure 'unique Id' is correctly named in the dataframe
#             'prediction': predicted_string
#         })

# # Create DataFrame for predictions
# predictions_df = pd.DataFrame(predictions)

In [127]:
import torch
from tqdm import tqdm
from PIL import Image
import pandas as pd
import torchvision.transforms as transforms

# Assuming 'device' and 'model' are defined earlier, for example:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)  # Move model to the device

predictions = []

# Set the model to evaluation mode
model.eval()

with torch.no_grad():  # No need to calculate gradients during inference
    for _, row in tqdm(df1.iterrows(), total=len(df1), desc="Inference"):
        image_path = row['image_path']
        
        # Open the image as grayscale
        image = Image.open(image_path).convert("L")  # 'L' mode for grayscale
        
        # Transform the image to tensor format
        transform = transforms.ToTensor()
        image_tensor = transform(image)  # Shape: [1, H, W] for grayscale
        
        # Stack the grayscale image to 3 channels to match model input
        image_stacked = torch.cat([image_tensor] * 3, dim=0)  # Shape: [3, H, W]

        # Apply threshold transformation
        mask = image_stacked < 0.6875
        image_stacked[mask] = 0.0
        image_stacked[~mask] = 1.0

        # Reshape the image for batch processing
        image_stacked = image_stacked.unsqueeze(0)  # Add batch dimension, Shape: [1, 3, H, W]

        # Process the image using the processor without rescaling (as per your requirement)
        pixel_values = processor(image_stacked, return_tensors="pt", do_rescale=False).pixel_values.to(device)
        
        # Generate the output using the model
        outputs = model.generate(pixel_values)

        # Decode the generated output to get the predicted string
        predicted_string = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Append the results to predictions list
        predictions.append({
            'unique Id': row['unique Id'],  # Ensure 'unique Id' matches your column name
            'prediction': predicted_string
        })

# Create a DataFrame for storing predictions
predictions_df = pd.DataFrame(predictions)


Inference:   0%|          | 0/1001 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Inference: 100%|██████████| 1001/1001 [02:54<00:00,  5.74it/s]


In [128]:
predictions_df["prediction"]

0            ro su saber reis Virtuo tiene y
1          tos juicios de darne tiempo, para
2           que en vuestra educación lo mani
3       sestalle, pues me hallo tantos mesos
4           ña astoria En esta casa a día. E
                        ...                 
996         En sus fistras. En sus Nacionals
997         También le nacieron hijos á Sem,
998                    de todos los hijos de
999            estos fueron hijos de locián.
1000                 son las familias de Noé
Name: prediction, Length: 1001, dtype: object

In [137]:
real_texts = train_df['transcription'].tolist()  # Assuming 'transcription' is the column name

In [154]:
from transformers import BartForConditionalGeneration, BartTokenizer
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
import torch
from sklearn.model_selection import train_test_split

# Define your custom Dataset class
class PredictionLabelDataset(Dataset):
    def __init__(self, predictions, real_texts, tokenizer, max_length=128):
        self.predictions = predictions
        self.real_texts = real_texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.predictions)

    def __getitem__(self, idx):
        prediction = str(self.predictions[idx])
        real_text = str(self.real_texts[idx])
        
        # Tokenize inputs and outputs (real_text as labels)
        input_encoding = self.tokenizer(prediction, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        label_encoding = self.tokenizer(real_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

        # Replace padding tokens in labels with -100, which is ignored by loss function
        labels = label_encoding['input_ids'].squeeze()
        labels[labels == tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),  # Remove batch dimension
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'labels': labels  # Labels are real text tokenized
        }

# Load your data (ensure your dataframe has 'predictions' and 'transcription' columns)
predictions = predictions_df['prediction'].tolist()  
real_texts = df1['transcription'].tolist()  

# Initialize the tokenizer and model for BART
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# Split the data into training and validation sets
train_predictions, val_predictions, train_real_texts, val_real_texts = train_test_split(
    predictions, real_texts, test_size=0.1, random_state=42
)

# Create Dataset instances for training and validation
train_dataset = PredictionLabelDataset(train_predictions, train_real_texts, tokenizer)
val_dataset = PredictionLabelDataset(val_predictions, val_real_texts, tokenizer)

# Create DataLoaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Load the BART model for conditional generation
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    # Training phase
    for batch in train_loader:
        batch = {key: value.to(device) for key, value in batch.items()}
        
        # Forward pass
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")

    # Validation phase
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = {key: value.to(device) for key, value in batch.items()}
            
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
            val_loss += outputs.loss.item()
    
    print(f"Validation Loss: {val_loss/len(val_loader)}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_bart_model")
tokenizer.save_pretrained("fine_tuned_bart_model")




Epoch 1/3, Loss: 1.2018372458324098
Validation Loss: 0.9980443886348179
Epoch 2/3, Loss: 0.7945075960535752
Validation Loss: 0.9059946196419852
Epoch 3/3, Loss: 0.6450232562788746
Validation Loss: 0.9090357933725629


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('fine_tuned_bart_model/tokenizer_config.json',
 'fine_tuned_bart_model/special_tokens_map.json',
 'fine_tuned_bart_model/vocab.json',
 'fine_tuned_bart_model/merges.txt',
 'fine_tuned_bart_model/added_tokens.json')

In [156]:
model1 = model 

In [159]:
# Now, test the model and generate new predictions
test_predictions = predictions_df1['prediction'].tolist()  # Your test set predictions
test_dataset = PredictionLabelDataset(test_predictions, [''] * len(test_predictions), tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model1.eval()
corrected_predictions = []

with torch.no_grad():
    for batch in test_loader:
        # Ensure the inputs are properly converted to tensors and moved to the correct device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        # Generate corrected predictions
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Decode the generated tokens into human-readable text
        decoded_preds = [
            tokenizer.decode(pred, skip_special_tokens=True) 
            for pred in predictions
        ]
        
        corrected_predictions.extend(decoded_preds)

# The 'corrected_predictions' now hold the model's new predictions
print(corrected_predictions)


['otro feñor por fus recibirimos y occilim,imimimimim,,,,', 'cos juyzios de dame tiempo, para,,,,,,,,,,,,,,,,,,,,,,,', 'que en vueltra educacion lo mani.,,,,', 'feñale, pues me hallo tantos me des,,,,,aleale,,,,ale,,,alealealealealealealeale,,,,aleale,,\xa0\xa0\xa0\xa0', 'ha tendida en esta camz, a vna en,,,,,,,,,,,,,,,,,,', 'feradad tal, que de de de fis princi.........................................,,,,,,ad,,,adadad,,,,..,,..............', 'g me amenaza con la muerte, y,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,', 'yn elra orlandad en ciemos años,,,,,,,,,,,,,,,,,,', 'ali pues como os di al mundo con,,,,,,,,,,,,,,,,,\xa0\xa0\xa0oo', 'llolores, con los de Can peñofos ac.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,..', 'cidences, os procuro en caminac,,,,,,,,,,,,,,,,,,,,,,,,,,,,,', 'aora a Dios, y que renazcays de mia,,,,,, de,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\xa0\xa0', 'para el, por medio de la virud, a,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,', 'que os exporto con estos documenoo,,,....', 'cos ( que he recogi

In [148]:
predictions = []

# Set the model to evaluation mode
model.eval()

# Ensure the model is moved to the correct device
model.to(device)

with torch.no_grad():  # No need to calculate gradients during inference
    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Inference"):
        image_path = row['image_path']
        
        # Open the image as grayscale
        image = Image.open(image_path).convert("L")  # 'L' mode for grayscale
        
        # Transform the image to tensor format
        transform = transforms.ToTensor()
        image_tensor = transform(image)  # Shape: [1, H, W] for grayscale
        
        # Stack the grayscale image to 3 channels to match model input
        image_stacked = torch.cat([image_tensor] * 3, dim=0)  # Shape: [3, H, W]

        # Apply threshold transformation
        mask = image_stacked < 0.6875
        image_stacked[mask] = 0.0
        image_stacked[~mask] = 1.0

        # Reshape the image for batch processing
        image_stacked = image_stacked.unsqueeze(0)  # Add batch dimension, Shape: [1, 3, H, W]

        # Process the image using the processor without rescaling (as per your requirement)
        pixel_values = processor(image_stacked, return_tensors="pt", do_rescale=False).pixel_values.to(device)
        
        # Generate the output using the model
        outputs = model.generate(pixel_values)

        # Decode the generated output to get the predicted string
        predicted_string = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Append the results to predictions list
        predictions.append({
            'unique Id': row['unique Id'],  # Ensure 'unique Id' matches your column name
            'prediction': predicted_string
        })

# Create a DataFrame for storing predictions
predictions_df1 = pd.DataFrame(predictions)


Inference:   0%|          | 0/168 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1338: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Inference: 100%|██████████| 168/168 [00:33<00:00,  5.07it/s]


In [158]:
predictions_df1

,unique Id,prediction
0,P_1_L_1,otro feñor por fus recibisimos y occil
1,P_1_L_2,"cos juyzios de dame tiempo, para"
2,P_1_L_3,que en vueltra educacion lo mani
3,P_1_L_4,"feñale, pues me hallo tantos me des"
4,P_1_L_5,"ha tendida en esta camz, a vna en"
...,...,...
163,P_7_L_20,"Ciceron del Noble, y creed, que"
164,P_7_L_21,don calentos de que le aueys de dar
165,P_7_L_22,"cuenta muy el trecha, y cargo con"
166,P_7_L_23,que os ha obligado a fer defendor


In [160]:
# Assuming corrected_predictions is the list of new predictions generated
predictions_df1['corrected_prediction'] = corrected_predictions

# Check if the new column is added correctly
print(predictions_df1.head())


  unique Id                              prediction  \
0   P_1_L_1  otro feñor por fus recibisimos y occil   
1   P_1_L_2        cos juyzios de dame tiempo, para   
2   P_1_L_3        que en vueltra educacion lo mani   
3   P_1_L_4     feñale, pues me hallo tantos me des   
4   P_1_L_5       ha tendida en esta camz, a vna en   

                                corrected_prediction  
0  otro feñor por fus recibirimos y occilim,imimi...  
1  cos juyzios de dame tiempo, para,,,,,,,,,,,,,,...  
2              que en vueltra educacion lo mani.,,,,  
3  feñale, pues me hallo tantos me des,,,,,aleale...  
4  ha tendida en esta camz, a vna en,,,,,,,,,,,,,...  


In [ ]:
def add_quotes(value):
    # Check if the value is not already enclosed in double quotes
    if not (isinstance(value, str) and value.startswith('"') and value.endswith('"')):
        return f'"{value}"'  # Add double quotes
    return value 

In [ ]:
def remove_quotes(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Remove leading and trailing double quotes
        return value.strip('"')
    return value

In [ ]:
predictions_df['prediction']=predictions_df['prediction'].apply(remove_quotes)
print(predictions_df)

In [165]:
# Drop the 'prediction' column from the dataframe
predictions_df1 = predictions_df1.drop(columns=['prediction'])

# Check if the column is dropped correctly
print(predictions_df1.head())
#


KeyError: "['prediction'] not found in axis"

In [166]:
predictions_df1 = predictions_df1.rename(columns={'corrected_prediction': 'prediction'})

# Check if the column is renamed correctly
print(predictions_df1.head())

  unique Id                                         prediction
0   P_1_L_1  otro feñor por fus recibirimos y occilim,imimi...
1   P_1_L_2  cos juyzios de dame tiempo, para,,,,,,,,,,,,,,...
2   P_1_L_3              que en vueltra educacion lo mani.,,,,
3   P_1_L_4  feñale, pues me hallo tantos me des,,,,,aleale...
4   P_1_L_5  ha tendida en esta camz, a vna en,,,,,,,,,,,,,...


In [167]:
predictions_df1.to_csv('submission.csv', index=False)

In [168]:
predictions_df1.head()

,unique Id,prediction
0,P_1_L_1,"otro feñor por fus recibirimos y occilim,imimi..."
1,P_1_L_2,"cos juyzios de dame tiempo, para,,,,,,,,,,,,,,..."
2,P_1_L_3,"que en vueltra educacion lo mani.,,,,"
3,P_1_L_4,"feñale, pues me hallo tantos me des,,,,,aleale..."
4,P_1_L_5,"ha tendida en esta camz, a vna en,,,,,,,,,,,,,..."
